In [1]:
import requests
import pandas as pd

# Login

In [2]:
# roda acesso, api/
url = "http://127.0.0.1:9000/api/".replace("\\","/")

In [3]:
username="zeladorx@zeladorx.com.br"
password="Amendoin123"

body = {
    "username":username,
    "password":password
}

In [4]:
response = requests.post("http://127.0.0.1:9000/api/login", body)

In [5]:
# dados do usuário logado
response.json()

# {
#     'token': '171db50f8cc06dae8ae7338bdc5048ddc97f6784', #token de acesso
#     'id_random': '6uW6BnjPFHyd', #id randomico unico
#     'user_id': 1, #id de chave primaria
#     'username': 'zeladorx', #username
#     'email': 'zeladorx@zeladorx.com.br', #email
#     'is_superuser': True, #é superusario do ecossitema contratado
# }

{'token': '171db50f8cc06dae8ae7338bdc5048ddc97f6784',
 'id_random': '6uW6BnjPFHyd',
 'user_id': 1,
 'username': 'zeladorx',
 'email': 'zeladorx@zeladorx.com.br',
 'is_superuser': True}

# Auth

In [6]:
# todas as rotas precisam do tokem para ser acessada
headers = {
    "Authorization": f"Token {response.json()['token']}"
}

# Fato servico

In [7]:
# retorna todas os acompanhamentos ja criados
# codigo de permissão necessário --> 390: Pode visualizar o detalhamento de serviços
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/ListFatoServicoJardinagem/", headers=headers)
response.json()
df = pd.DataFrame(response.json())
display(df)

""


## Criar novo acompanhemto

In [8]:
## As colunas ServicoJardinagemAgendado são chaves estrangeiras e precisam estar devidamente configuradas
## sendo obriagatorios

In [9]:
# Lista de empresas que podem ser enviadas pelo usuario
# nao precisa de permissao
print("Serviços agendados")
response = requests.get("http://127.0.0.1:9000/api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/", headers=headers)
response.json()
df = pd.DataFrame(response.json()['results'])
display(df)
print()

Serviços agendados


,id,id_random,DescricaoDoServico,DataDeInicio,DataDeConclusao,TipoServico,Areas,ServicosEscalados,ColaboradoresEscalados,foto_solicitacao,foto_entrega,dias_diferenca,novo_status
0,10,P1n687HSearU,steste,2025-08-16T23:19:00-03:00,2025-08-18T23:19:00-03:00,Regular,12,"[1, 4]",[6],None,None,0,Próximo


In [12]:
body = {
    'Servico': 10, #vem de api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/
    'data_hora_chegada_na_area': "2025-08-08T13:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'data_hora_retorno_area': "2025-08-08T18:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'Gerente': 6, # vem de api/gerentes/ListGerenteJardinagemFromForms/
}


In [13]:
# codigo de permissão necessário --> 324: Pode acompanhar serviços agendados
# essa rota rece o id random do servico que sera crado o acompanhamento, deve ser validado jnto com o id
response = requests.post("http://127.0.0.1:9000/api/servicos/fato/CreateFatoServicoJardinagem/P1n687HSearU/", body, headers=headers)
response.json()

{'success': True,
 'message': 'FatoServicoJardinagem criado com sucesso.',
 'data': {'id': 6,
  'id_random': 'BsYKHA7RXFI9',
  'Servico': 10,
  'data_hora_chegada_na_area': '2025-08-08T13:57:00-03:00',
  'data_hora_retorno_area': '2025-08-08T18:57:00-03:00',
  'Gerente': 6}}

# Detalhes acompanhamento

In [15]:
# essa rota recebe o id random da acompanahemnto como parametro de url, consultar em api/servicos/fato/ListFatoServicoJardinagem/
# detalhes do acompanhemtno criado anteriormente, obeserve o id_random
# http://127.0.0.1:9000/api/servicos/fato/FatoServicoJardinagemDetail/id_random/
# codigo de permissão necessário --> 402: Pode visualizar checklists
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/FatoServicoJardinagemDetail/BsYKHA7RXFI9/", headers=headers)
response.json()

{'id': 6,
 'id_random': 'BsYKHA7RXFI9',
 'Servico': 10,
 'data_hora_chegada_na_area': '2025-08-08T13:57:00-03:00',
 'data_hora_retorno_area': '2025-08-08T18:57:00-03:00',
 'Gerente': 6}

# Atualizar acompanhamento

In [23]:
# as chaves body da atualização é exatamente igual o body da criacao
# atualiando a descrição para acompanhamento teste
body = {
    'Servico': 10, #vem de api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/
    'data_hora_chegada_na_area': "2025-08-08T13:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'data_hora_retorno_area': "2025-08-08T18:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'Gerente': 6, # vem de api/gerentes/ListGerenteJardinagemFromForms/
}

In [24]:
# essa rota recebe o id random do checklist como parametro de url, consultar em /api/checklist/ListCheckListJardinagem
# atualizando o checklist criada anteriormente, obeserve o id_random
# codigo de permissão necessário --> 401: Pode editar checklists
response = requests.put("http://127.0.0.1:9000/api/servicos/fato/FatoServicoJardinagemUpdate/BsYKHA7RXFI9/", body, headers=headers)
response.json()

{'success': True,
 'message': 'FatoServicoJardinagem atualizado com sucesso.',
 'data': {'id': 6,
  'id_random': 'BsYKHA7RXFI9',
  'Servico': 10,
  'data_hora_chegada_na_area': '2025-08-08T13:57:00-03:00',
  'data_hora_retorno_area': '2025-08-08T18:57:00-03:00',
  'Gerente': 6}}

# Se deletar o acompanhamento

In [26]:
# essa rota recebe o id random do acompanhemtno como parametro de url, consultar em api/servicos/fato/ListFatoServicoJardinagem/
# codigo de permissão necessário --> 327 Pode excluir serviços concluidos
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/IfDeleteFatoServicoJardinagem/BsYKHA7RXFI9/", headers=headers)
response.json()

{'objeto_principal': 'FatoServicoJardinagem object (6)',
 'resumo': {'FatoServicoJardinagem': 1},
 'objetos_em_cascata': ['- FatoServicoJardinagem object (6)']}

# Deleção do acompanhamento

In [28]:
# essa rota recebe o id random do acompanhemtno como parametro de url, consultar em api/servicos/fato/ListFatoServicoJardinagem/
# codigo de permissão necessário --> 327 Pode excluir serviços concluidos
response = requests.delete("http://127.0.0.1:9000/api/servicos/fato/DeleteFatoServicoJardinagem/BsYKHA7RXFI9/", headers=headers)
response.json()

{'detail': "FatoServicoJardinagem 'FatoServicoJardinagem object (6)' foi deletado com sucesso."}

# Endpoints Auxiliares

# acompanhamentos associado a um servico especifico

In [24]:
# essa rota recebe  id random de um servico como parametro, obtido em api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/
# codigo de permissão 390: Pode visualizar o detalhamento de serviços
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/ListFatoServicoJardinagemByservico/vfkNAWVVI4BJ/", headers=headers)
response.json()

[{'id_acompanhamento': 3,
  'id_random_acompanhamento': 'nB0ZjXJV19VN',
  'id_agendamento': 3,
  'id_random_agendamento': 'vfkNAWVVI4BJ',
  'colaboradores_chamados_id': 5,
  'colaboradores_chamados_id_random': 'K4341ZakSLwN',
  'colaboradores_chamados': 'markos',
  'data_hora_chegada': '2025-08-01 02:00',
  'data_hora_retorno': '2025-08-01 11:00',
  'tempo_na_area': '09:00:00'}]